<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 22: Markov Chain Monte Carlo - Part I</h1>



<a name='section_23_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L22.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_23_1">L22.1 Markov Chain MC</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_23_1">L22.1 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_23_2">L22.2 Understanding some details</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_23_2">L22.2 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_23_3">L22.3 A more realistic Markov Chain MC</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_23_3">L22.3 Exercises</a></td>
    </tr>
</table>

<h3>Learning Objectives</h3>

Building upon our work with Monte-Carlo and simulation, we now look at the scenario when our simulation is not perfect. Nonetheless, we have ways to check whether it is perfect, and we will adjust the simulation on the fly to correct it. This approach is amenable to fitting data with complex models as well. This leads to the Markov-Chain Process that we will elaborate on in detail in this lecture. 

<h3>Slides</h3>

You can access the slides related to this lecture at the following link: <a href="https://github.com/mitx-8s50/slides/raw/main/module3_slides/L23_slides.pdf" target="_blank">L23 Slides</a>

<h3>Data</h3>

Download the data from the following directory. Reference below:

>description: EPICA Dome C Ice Core 800KYr Deuterium Data and Temperature Estimates.<br>
>source:  https://www.ncei.noaa.gov/pub/data/paleo/icecore/antarctica/epica_domec/edc3deuttemp2007.txt<br>
>attribution: Jouzel, J., et al.  2007.
EPICA Dome C Ice Core 800KYr Deuterium Data and Temperature Estimates. 
IGBP PAGES/World Data Center for Paleoclimatology 
Data Contribution Series # 2007-091.
NOAA/NCDC Paleoclimatology Program, Boulder CO, USA.

In [ ]:
#>>>RUN: L22.0-runcell00

!git init
!git remote add -f origin https://github.com/mitx-8s50/nb_LEARNER/
!git config core.sparseCheckout true
!echo 'data/L23' >> .git/info/sparse-checkout
!git pull origin main


<h3>Installing Tools</h3>

Before we do anything, let's make sure we install the tools we need.

In [ ]:
#>>>RUN: L22.0-runcell01

!pip install corner
!pip install lmfit
!pip install bilby
!pip install gwpy lalsuite

<h3>Importing Libraries</h3>

Before beginning, run the cell below to import the relevant libraries for this notebook. 

In [ ]:
#>>>RUN: L22.0-runcell01

import imageio
from PIL import Image

import lmfit
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import csv
import math
from scipy import optimize as opt
from scipy import stats
import matplotlib.cm as cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import corner

import bilby
import scipy.signal as sig
from bilby.gw.source import lal_binary_black_hole
from bilby.gw.conversion import convert_to_lal_binary_black_hole_parameters

<h3>Setting Default Figure Parameters</h3>

The following code cell sets default values for figure parameters.


In [ ]:
#>>>RUN: L22.0-runcell02

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title

<a name='section_23_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L22.1 Markov Chain Monte Carlo </h2>  

| [Top](#section_23_0) | [Previous Section](#section_23_0) | [Exercises](#exercises_23_1) | [Next Section](#section_23_2) |


*The material in this section is discussed in the videos **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.3x+3T2023/block-v1:MITxT+8.S50.3x+3T2023+type@sequential+block@seq_LS22/block-v1:MITxT+8.S50.3x+3T2023+type@vertical+block@vert_LS22_vid1" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Overview</h3>

Markov Chain Monte Carlo (MCMC) is a computational method used for sampling from complex probability distributions, especially when direct sampling or analytical methods are not feasible. Its key components include the following:

1. A **Markov Chain** is a sequence of random variables where the probability distribution of each variable depends only on the state of the previous one. The "Markov" property means that the future state depends only on the current state, not on the sequence of events that preceded it.


2. The key to MCMC's success is satisfying the **detailed balance** condition. In simple terms, the probability of transitioning from state A to state B must be the same as transitioning from state B to state A. This ensures that the Markov chain converges to the desired distribution.


3. The MCMC method has an initial phase, known as the **burn-in** period, which allows the Markov chain to explore the state space and reach a region where the samples are representative of the target distribution. Samples obtained during the burn-in phase are typically discarded.


4. Once the Markov Chain is considered to be in a stationary state, subsequent **samples are collected.** These samples can be used to approximate the desired distribution and estimate parameters of interest.

<h3>A simple example</h3>

To understand Markov Chain Monte Carlo, we are going to sample a normal distribution, and then **we are going to use a Markov Chain MC to fit this normal distribution.** While this seems like a somewhat trivial example, the point here is to illustrate a basic example of how MCMC attempts to model parameters. 

Later on, we will see that, while MCMC somewhat painstakingly extracts parameters for fits, it can be used to perform modeling of complex processes that other approaches cannot do because they lack the same stability.

We will start our Markov Chain MC by creating a toy dataset, sampling 1000 events from a Gaussian with a mean of 50 and width of 20.

In [ ]:
#>>>RUN: L22.1-runcell01

nsample=1000
values=np.random.normal(50,20,nsample)
fig = plt.figure(figsize=(10,10))
plt.hist(values,bins=50)
plt.xlabel("x")
plt.ylabel("N")


Normally, we would get the parameters describing this distribution by fitting a Gaussian. This would involve minimizing the loss:

\begin{eqnarray}
\log\left(\mathcal{L}\right) & = & \sum_{i} \log\left(\frac{1}{\sqrt{2\pi\sigma^{2}}} e^{-\dfrac{\left(\mu-x_{i}\right)^2}{\sigma^2}}\right)\\
& = & \sum_{i}\left(\frac{x_{i}-\mu}{\sigma}\right)^2 - \frac{1}{2}\log\left(2\pi\sigma^2\right)
\end{eqnarray}

However, what if we don't know how to compute the gradient for the above loss function? In that case, we can just compute the Log Likelihood, $\log(p_{i})$ for two randomly chosen sets of parameters, and keep the second set if $\log(p_{i})$ increases. If it does not increase, we will resample the parameters with some sampling policy and try again.

Now, what we are going to do is try to extract the mean and RMS of this distribution of data. The way we will do this is with a multi-step process, whereby we:

 * Define a Likelihood $p_{i}$ for the agreement between the prediction and the data
 * Define a proposal distribution from which to sample to get best fit parameters
   * Note this proposal distribution can be anything (we'll play around with different options)
 * Sample the proposal distribution to get an initial guess of the parameters ($\vec{\theta}$)
 * Sample the proposal distribution again to get a second paramter set ($\vec{\theta^\prime}$)
   * Compute the ratio of the likelihoods of these two parameter sets to find a proposal weight $s_{\rm check}$:
$$
s_{\rm accept} = \frac{p_{\rm sampled} (\vec{\theta^\prime})}{p_{\rm current} (\vec{\theta})}
$$
  * If $s_{\rm accept}\gt 1$, accept $\vec{\theta^{\prime}}$ as the new parameter set, otherwise
    * Sample a uniform distribution between 0 and 1 $\rightarrow s_{\rm check}$
    * If  $s_{\rm accept} \gt s_{\rm check}$ accept $\vec{\theta^{\prime}}$ as the new parameter set
    * For example, if $s_{\rm accept} = 0.5$, we would keep $\vec{\theta^{\prime}}$ as the new "best" set of parameters 50\% of the time.
  * Repeat the above until the parameter set $\vec{\theta^{\prime}}$ converges
  * Once it equilibrates, use the final $\vec{\theta^{\prime}}$ to generate a posterior
  
This allows us to get a best fit to data without doing any actual fitting. Note that our likelihood doesn't need to be a true likelihood. It could be solving a differential equation, or minimizing some other thing. Also, it doesn't need to be differentiable, we just need something that gives us an indication of a better fit.

**Fitting the data**

Ok, let's now step through and try to fit the random data that we generated. First, we will define the Log Likelihood function shown above as the probability that we wish to maximize.

In [ ]:
#>>>RUN: L22.1-runcell02

def log_like_normal(x,data):
    #x[0]=mu, x[1]=sigma (new or current)
    #data = the observation
    return np.sum(np.log(stats.norm(x[0],x[1]).pdf(data)))


Now, what we want to do is choose a **Proposal Distribution** to sample $\vec{\theta^{\prime}}$. The key here is to make sure our proposal distribution covers the possible values for $\mu$ and $\sigma$ that match our data, otherwise we will never converge. One way to do this is to make our **Proposal Distribution** a function of our inputs.

As a first example, let's start with the current proposed values of the mean and width $\vec{\theta} = [\mu,\sigma]$ and generate $\vec{\theta^{\prime}}$ by sampling a flat distribution between $x_{i} - \sigma$ and  $x_{i} + \sigma$, where $x_{i}$ is either the current proposed values of $\mu$ or $\sigma$.

Try running the code cell below multiple times to see how the values of $\vec{\theta^{\prime}}$ and its associated likelihood vary. Notice that values of $\vec{\theta^{\prime}}$ closer to [50,20] haver larger (i.e. less negative) likelihoods.

In [ ]:
#>>>RUN: L22.1-runcell03

def proposal(x):
    #x[0] = mu, x[1]=sigma (new or current)
    x_new = np.array([0,0])
    x_new = np.random.uniform(x-x[1]*np.ones(x.shape),x+x[1]*np.ones(x.shape))
    return x_new

#Let's run a quick test
xinit=np.array([25,10])
likeOld=log_like_normal(xinit,values)
xtry=proposal(xinit)
likeNew=log_like_normal(xtry,values)
print("Init:",xinit, "Likelihood:",likeOld)
print("Try:" ,xtry , "Likelihood:",likeNew)

Now, we need to translate our likelihood to a p-value and convert it to an accept block to define how our parameters will migrate over time. Let's go ahead and define an accept block. Given that we are using $\log(\mathcal{L})$, we need to translate this from $\log(p)\rightarrow p$, which we can do just by exponentiating.

In other words (note, the notation below is different than what is presented in the corresponding video):

$$
s_{\rm accept} = \frac{p_{\rm new} (\vec{\theta^\prime})}{p_{\rm old} (\vec{\theta})} \\
s_{\rm accept} = e^{\large\left(\log\left(\mathcal{L}_{\rm new}\right) - \log\left(\mathcal{L}_{\rm old}\right)\right)}
$$

Finally, we can add constraints or priors to our sampled $\vec{\theta^{\prime}}$ values. This leads to a modified acceptance probability, where we inject priors into our fit by:

$$
\begin{align}
s_{\rm accept} &= \frac{p_{\rm new} (\vec{\theta^\prime}){\rm prior(\vec{\theta^\prime}})}{p_{\rm old} (\vec{\theta}){\rm prior(\vec{\theta}})} \\
&= e^{\large\left(\log\left(\mathcal{L}_{\rm new}\right) - \log\left(\mathcal{L}_{\rm old}\right)\right)}
\end{align}
$$

Note that we get the same final equation as before. The difference is that we can use constraints from the prior to reject some of the  $\vec{\theta^{\prime}}$ parameter sets that our sampling produces. As one example, code cell `L22.1-runcell04` rejects any samples which produce a negative value for sigma.

Try running multiple instances of code cell `L22.1-runcell03` followed by `L22.1-runcell04` to see which sets $\vec{\theta^{\prime}}$ are accepted.

In [ ]:
#>>>RUN: L22.1-runcell04

#note, the notation below is different than what is presented in the corresponding video


#Defines whether to accept or reject the new sample
def acceptance(likeOld, likeNew):
    if likeNew>likeOld:
        return True
    else:
        accept=np.random.uniform(0,1)
        return (accept < (np.exp(likeNew-likeOld)))

def prior(like,x):
    #Adjust the likelihood by the prior
    prior=1
    if(x[1] <=0):
        prior=0
    return like+np.log(prior) #log(1)=0 so nothing gets added, log(0)=-infinity so this case always gets rejected

print("Init Prior:",prior(likeOld,xinit),"Try Prior:",prior(likeNew,xtry))
print("Accept:",acceptance(prior(likeOld,xinit),prior(likeNew,xtry)))

Ok, now let's set up the full Markov-Chain Monte Carlo chain. By the way, there are many different Markov Chain Monte Carlo procedures. The one that we will implement below is known as the <a href="https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm" target="_blank">Metropolis-Hastings algorithm</a>, named after Nicholas Metropolis and W.K. Hastings.

Let's write it all out and see how it works. In this example, we run 15,000 iterations of finding and checking new parameter sets. Note that, as for the examples above, we are using starting values $\vec{\theta} = [25,10]$ which were deliberately chosen not to be too close or too far from the parameters used to generate the data. To see how sensitve the MCMC procedure is to this choice, try values that are farther away from the truth.

In [ ]:
#>>>RUN: L22.1-runcell05

def metropolis_hastings(iLikelihood,iPrior,iProposal,iAcceptance,xinit,data,niterations):
    x = xinit
    accepted = []
    rejected = []
    likelihood = []
    for i in range(niterations):
        x_new   =  iProposal(x)  
        likeOld = iLikelihood(x,data)
        likeNew = iLikelihood(x_new,data) 
        if (iAcceptance(iPrior(likeOld,x),iPrior(likeNew,x_new))):            
            x = x_new
            accepted.append(x_new)
            likelihood.append(likeNew)
        else:
            rejected.append(x_new)
    accepted = np.array(accepted)
    rejected = np.array(rejected)
    likelihood = np.array(likelihood)
    return accepted, rejected, likelihood

xinit = np.array([25,10])
accepted, rejected, likelihood = metropolis_hastings(log_like_normal,prior,proposal,acceptance,xinit,values,15000)

Now, let's plot the results:

In [ ]:
#>>>RUN: L22.1-runcell06

#PLOT SIGMA VALUES
fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(3,1,1)
print(rejected[0:50][0])

ax1.plot(rejected[0:50,1], 'rx', label='Rejected sigma',alpha=0.5)
ax1.plot(accepted[0:50,1], 'b.', label='Accepted sigma',alpha=0.5)
ax1.set_xlabel("Iteration")
ax1.set_ylabel("$\sigma$")
ax1.grid()
ax1.legend()


ax2 = fig.add_subplot(3,1,2)
#to_show=-accepted.shape[0]
ax2.plot( rejected[:,1], 'rx', label='Rejected sigma',alpha=0.5)
ax2.plot( accepted[:,1], 'b.', label='Accepted sigma',alpha=0.5)
ax2.set_xlabel("Iteration")
ax2.set_ylabel("$\sigma$")
ax2.grid()
ax2.legend()

ax3 = fig.add_subplot(3,1,3)
ax3.plot( likelihood, 'rx', label='Likelihood',alpha=0.5)
ax3.set_xlabel("Iteration")
ax3.set_ylabel("$\mathcal{L}$")

fig.tight_layout()
accepted.shape
plt.show()

In [ ]:
#>>>RUN: L22.1-runcell07

#PLOT MU VALUES
fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(3,1,1)
print(rejected[0:50][0])

ax1.plot(rejected[0:50,0], 'rx', label='Rejected mu',alpha=0.5)
ax1.plot(accepted[0:50,0], 'b.', label='Accepted mu',alpha=0.5)
ax1.set_xlabel("Iteration")
ax1.set_ylabel("$\mu$")
ax1.grid()
ax1.legend()


ax2 = fig.add_subplot(3,1,2)
ax2.plot( rejected[:,0], 'rx', label='Rejected mu',alpha=0.5)
ax2.plot( accepted[:,0], 'b.', label='Accepted mu',alpha=0.5)
ax2.set_xlabel("Iteration")
ax2.set_ylabel("$\mu$")
ax2.grid()
ax2.legend()

ax3 = fig.add_subplot(3,1,3)
ax3.plot( likelihood, 'rx', label='Likelihood',alpha=0.5)
ax3.set_xlabel("Iteration")
ax3.set_ylabel("$\mathcal{L}$")

fig.tight_layout()
accepted.shape
plt.show()

As you can see, all of the samples tried after less than about 50 iterations get rejected (this seems incredibly inefficient). Let's look at our best fit parameters.

In [ ]:
#>>>RUN: L22.1-runcell07

_,bins,_ = plt.hist(rejected[:,1],bins=35,density=True,label='reject',alpha=0.5)
_,bins,_ = plt.hist(accepted[:,1],bins=bins,density=True,label='accept',alpha=0.5)
_,bins,_ = plt.hist(accepted[-20:,1],bins=bins,density=True,label='accept(last 20)',alpha=0.5)
plt.xlabel("$\sigma$")
plt.ylabel("pdf")
plt.legend()
plt.show()

_,bins,_ = plt.hist(rejected[:,0],bins=35,density=True,label='reject',alpha=0.5)
_,bins,_ = plt.hist(accepted[:,0],bins=bins,density=True,label='accept',alpha=0.5)
_,bins,_ = plt.hist(accepted[-20:,0],bins=bins,density=True,label='accept(last 20)',alpha=0.5)
plt.xlabel("$\mu$")
plt.ylabel("pdf")
plt.legend()
plt.show()

print("Average and stdev of all accepted mu:",accepted[:,0].mean(),"+/-",accepted[:,0].std())
print("Average and stdev of all accepted sigma:",accepted[:,1].mean(),"+/-",accepted[:,1].std())

print("Average and stdev of last 20 accepted mu:",accepted[-20:,0].mean(),"+/-",accepted[-10:,0].std())
print("Average and stdev of last 20 accepted sigma:",accepted[-20:,1].mean(),"+/-",accepted[-10:,1].std())


Now, let's compare these results with those found using an analytic computation, as well as a regular fit.

In [ ]:
#>>>RUN: L22.1-runcell09

##Analytic
print("Analytic results for mean and standard deviation of the data:")
print("Mu:",values.mean(),"+/-",values.std()/np.sqrt(len(values)))
print("Sigma:",values.std(),"+/-",values.std()/np.sqrt(2.*len(values)))
print()

y,bin_edges=np.histogram(values,bins=35)
bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])
#lmfit
from lmfit.models import GaussianModel
model = GaussianModel()
params = model.make_params(center=25, amplitude=1, sigma=10)
result = model.fit(y, params, x=bin_centers,weights=1./np.sqrt(y+1))
result.plot()
print("Fitted results for mean (\"center\") and standard deviation (\"sigma\") of the data:")
print(result.fit_report())

Note that the results shown above use 3 different ways of calculating the uncertainties. For the MCMC, the uncertainties are simply the standard deviations of the accepted values (for either all of them or only the last 10). For the analytic calculations, the theoretical uncertainties in the mean and standard deviation for a sample of size $n$ are listed:

$$
\sigma_{mean}=\sigma/\sqrt{n}\\  
\sigma_{\sigma}=\sigma/\sqrt{2n}
$$

Finally, for the fit, the variations are found as a part of the fitting process itself.

Now, we can try to to improve the Markov Chain so that it doesn't take a ridiculous number of iterations to get to some level of convergence. The simplest approach to accomplish this is to change our sampling proposal. In the example above, we simply varied the parameters over a broad uniform distribution of values. In the example below, we will change the sampling to something much more optimal. Specifically, we will get new proposed means and sigmas by starting with the existing values and varying them by a small fraction of their uncertainties as listed above. We'll chose that fraction using a normal distribution of mean 0 and width 1. That way, we limit how far $\vec{\theta^\prime}$ can deviate from $\vec{\theta}$. The small deviations will allow us to slowly crawl to the best fit solution.

Once we are there, we will use this sampling procedure to build an uncertainty profile of the various parameters by continuing to vary the parameters and running the MCMC acceptance step. Parameters, for which the likelihood is better will always be accepted, and parameters where teh p-value is comparable, but worse to the previous set of parameters will be rejected following the MCMC sampling.   

In [ ]:
#>>>RUN: L22.1-runcell10

def proposal(x,n=1000):
    #x[0] = mu, x[1]=sigma (new or current)
    x_new     = x.copy()
    x_new[0]  = x[0] + np.random.randn()*x[1]/np.sqrt(n)
    x_new[1]  = x[1] + np.random.randn()*x[1]/np.sqrt(2*n)
    return x_new

xinit = np.array([25.,10.])
accepted, rejected,likelihood = metropolis_hastings(log_like_normal,prior,proposal,acceptance,xinit,values,15000)

Again, let's plot the behavior:

In [ ]:
#>>>RUN: L22.1-runcell11

plt.plot(likelihood)
plt.xlabel("Iteration")
plt.ylabel("Likelihood")
plt.ylim(-4500,-4300)
plt.show()

_,bins,_ = plt.hist(rejected[:,1],bins=35,density=True,label='reject',alpha=0.5)
_,bins,_ = plt.hist(accepted[:,1],bins=bins,density=True,label='accept',alpha=0.5)
_,bins,_ = plt.hist(accepted[-20:,1],bins=bins,density=True,label='accept(last 20)',alpha=0.5)
plt.xlabel("$\sigma$")
plt.ylabel("pdf")
plt.legend()
plt.show()

_,bins,_ = plt.hist(rejected[:,0],bins=35,density=True,label='reject',alpha=0.5)
_,bins,_ = plt.hist(accepted[:,0],bins=bins,density=True,label='accept',alpha=0.5)
_,bins,_ = plt.hist(accepted[-20:,0],bins=bins,density=True,label='accept(last 20)',alpha=0.5)
plt.xlabel("$\mu$")
plt.ylabel("pdf")
plt.legend()
plt.show()

print("Average and stdev of all accepted mu:",accepted[:,0].mean(),"+/-",accepted[:,0].std())
print("Average and stdev of all accepted sigma:",accepted[:,1].mean(),"+/-",accepted[:,1].std())

print("Average and stdev of last 1000 accepted mu:",accepted[-1000:,0].mean(),"+/-",accepted[-1000:,0].std())
print("Average and stdev of last 1000 accepted sigma:",accepted[-1000:,1].mean(),"+/-",accepted[-1000:,1].std())


fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(2,1,1)
ax.plot(rejected[0:150,1], 'rx', label='Rejected',alpha=0.5)
ax.plot(accepted[0:150,1], 'b.', label='Accepted',alpha=0.5)
ax.set_xlabel("Iteration")
ax.set_ylabel("$\sigma$")
ax.grid()
ax.legend()


ax2 = fig.add_subplot(2,1,2)
#to_show=-accepted.shape[0]
ax2.plot( rejected[:,1], 'rx', label='Rejected',alpha=0.5)
ax2.plot( accepted[:,1], 'b.', label='Accepted',alpha=0.5)
ax2.set_xlabel("Iteration")
ax2.set_ylabel("$\sigma$")
ax2.grid()
ax2.legend()



fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(2,1,1)
ax.plot(rejected[0:150,0], 'rx', label='Rejected',alpha=0.5)
ax.plot(accepted[0:150,0], 'b.', label='Accepted',alpha=0.5)
ax.set_xlabel("Iteration")
ax.set_ylabel("$\mu$")
ax.grid()
ax.legend()


ax2 = fig.add_subplot(2,1,2)
#to_show=-accepted.shape[0]
ax2.plot( rejected[:,0], 'rx', label='Rejected',alpha=0.5)
ax2.plot( accepted[:,0], 'b.', label='Accepted',alpha=0.5)
ax2.set_xlabel("Iteration")
ax2.set_ylabel("$\mu$")
ax2.grid()
ax2.legend()



Now our best fit looks much better in the sense that we have many accepts. In fact we have so many accepts that we have more than our rejects, this is why we see the acceptance list is longer than the rejected list. Note that the iterations are counted separate for reject and accept, so we have a total of 15k iterations, which is the sum of the roughly 8k accepted iteartions and 7k rejects. Clearly, we see that our accepted rate is quite high. You can see that there is an early stage before we get to equilibration. This stage is typically referred to as the burn-in.


<a name='exercises_23_1'></a>     

| [Top](#section_23_0) | [Restart Section](#section_23_1) | [Next Section](#section_23_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-22.1.1</span>

What happens when your proposal function gets too narrow or too wide? Edit the function `proposal_1` in the starting code shown below to try the fit with a range for finding $\vec{\theta^{\prime}}$ (called `x_new` in the code) that is a factor of 10 smaller than that used in code cell `L22.1-runcell09`, and edit `proposal_2` to try with a range 10X larger. Let's focus specifically on the uncertainties in the final values of the parameters as calculated using the standard deviation of the accepted parameter sets near the end of the iterations. What changes do you observe? You could also plot the distributions of accepted and rejected points vs. iteration, to gain further insight (as done above).

Try several runs to check whether or not you see a clear trend.

A) The uncertainties that are computed do not change at all.\
B) The uncertainties change from run to run, but do not seem to have a consistent dependence on the proposal that is used.\
C) The uncertainties depend on the proposal that is used, and seem to be positively correlated, meaning that the computed uncertainty increases when a wider proposal function is used.\
D) The uncertainties depend on the proposal that is used, and seem to be negatively correlated (anticorrelated), meaning that the computed uncertainty decreases when a wider proposal function is used.


In [ ]:
#>>>EXERCISE: L22.1.1

def proposal_1(x,n=2000):
    #YOUR CODE HERE
    return x_new

xinit = np.array([25.,10.])
accepted_1, rejected_1, _ = metropolis_hastings(log_like_normal,prior,proposal_1,acceptance,xinit,values,5000)

print("1/10 original uncertainty")
print("Length of last 1000 accepted:",len(accepted_1[-1000:,1]))
print("Average and stdev of last 1000 accepted mu:",accepted_1[-1000:,0].mean(),"+/-",accepted_1[-1000:,0].std())
print("Average and stdev of last 1000 accepted sigma:",accepted_1[-1000:,1].mean(),"+/-",accepted_1[-1000:,1].std())
print()

def proposal_2(x,n=2000):
    #YOUR CODE HERE
    return x_new

xinit = np.array([25.,10.])
accepted_2, rejected_2, _ = metropolis_hastings(log_like_normal,prior,proposal_2,acceptance,xinit,values,5000)


print("10X original uncertainty")
print("Length of last 1000 accepted:",len(accepted_2[-1000:,1]))
print("Average and stdev of last 1000 accepted mu:",accepted_2[-1000:,0].mean(),"+/-",accepted_2[-1000:,0].std())
print("(Average and stdev of last 1000 accepted sigma:",accepted_2[-1000:,1].mean(),"+/-",accepted_2[-1000:,1].std())
print()


#PLOT REJECTED VS. ACCEPTED
#YOUR CODE HERE (if you want)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-22.1.2</span>

Let's now see what happens when we use a smaller number of events to calculate the parameters and uncertainties. Using your definitions for `proposal_1` and `proposal_2` that you had above, change the final number of events you sample to take just the last 100 events? Play with the results, which answer gives you the correct result (note the cramer-rao bound holds).

A) The uncertainties on both are fine.\
B) Taking the larger range is good for a wide proposal, and the smaller range is good for a narrow proposal. \
C) Taking the larger range is good for a narrow proposal, and the smaller range is good for a wide proposal. \
D) A smaller range is needed for the wide proposal because of turn on, while the narrow proposal gives uncertainties that are too small (below the Cramer-Rao bound) because it is sampling parameters with too small a proposal distribution. 



In [ ]:
#>>>EXERCISE: L22.1.2

def proposal_1(x,n=2000):
    #YOUR CODE HERE
    return x_new

xinit = np.array([25.,10.])
accepted_1, rejected_1, _ = metropolis_hastings(log_like_normal,prior,proposal_1,acceptance,xinit,values,5000)

print("1/10 original uncertainty")
print("Length of last 100 accepted:",len(accepted_1[-###:,1]))
print("Average and stdev of last 100 accepted mu:",accepted_1[-###:,0].mean(),"+/-",accepted_1[-###:,0].std())
print("Average and stdev of last 100 accepted sigma:",accepted_1[-###:,1].mean(),"+/-",accepted_1[-###:,1].std())
print()

def proposal_2(x,n=2000):
    #YOUR CODE HERE
    return x_new

xinit = np.array([25.,10.])
accepted_2, rejected_2, _ = metropolis_hastings(log_like_normal,prior,proposal_2,acceptance,xinit,values,5000)

print("10X original uncertainty")
print("Length of last 100 accepted:",len(accepted_2[-###:,1]))
print("Average and stdev of last 100 accepted mu:",accepted_2[-###:,0].mean(),"+/-",accepted_2[-###:,0].std())
print("(Average and stdev of last 100 accepted sigma:",accepted_2[-###,1].mean(),"+/-",accepted_2[-###:,1].std()))
                   
                                                                  
#PLOT REJECTED VS. ACCEPTED
#YOUR CODE HERE (if you want)

<a name='section_23_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L22.2 Understanding Some Details </h2>  

| [Top](#section_23_0) | [Previous Section](#section_23_1) | [Exercises](#exercises_23_2) | [Next Section](#section_23_3) |

*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.3x+3T2023/block-v1:MITxT+8.S50.3x+3T2023+type@sequential+block@seq_LS22/block-v1:MITxT+8.S50.3x+3T2023+type@vertical+block@vert_LS22_vid2" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Overview</h3>

Now a big component of why Markov Chain Monte Carlo is so powerful when compared with traditional fitting is the fact that we get a sequence of Monte Carlo events that give lots of details about our best fit. As just one example, these MC events allow us to look at the correlation of the final parameters, and help us to understand the space of allowed convergence. This flexibility due to the nature of Monte Carlo makes this approach particularly compelling.

For example, let's take our previous Gaussian fit and look at how the parameters start to converge to the best fit.

In [ ]:
#>>>RUN: L22.2-runcell01

#This cell plots results from our original run, which produced the array `accepted`

fig = plt.figure(figsize=(10,20))
ax = fig.add_subplot(3,1,1)
ax.plot(accepted[:50,0], accepted[:50,1], label="Path")
ax.plot(accepted[:50,0], accepted[:50,1], 'b.', label='Accepted(First 50)')
ax.plot(rejected[:50,0], rejected[:50,1], 'rx', label='Rejected(First 50)')
ax.set_xlabel("$\mu$")
ax.set_ylabel("$\sigma$")
ax.legend()


ax = fig.add_subplot(3,1,2)
ax.plot(accepted[:,0], accepted[:,1], label="Path")
ax.plot(accepted[:,0], accepted[:,1], 'b.', label='Accepted(Full)',alpha=0.3)
ax.plot(rejected[:,0], rejected[:,1], 'rx', label='Rejected(Full)',alpha=0.3)
ax.set_xlabel("$\mu$")
ax.set_ylabel("$\sigma$")
ax.legend()
ax.set_title("") 

to_show=50
ax = fig.add_subplot(3,1,3)
ax.plot(accepted[-to_show:,0], accepted[-to_show:,1], label="Path")
ax.plot(accepted[-to_show:,0], accepted[-to_show:,1], 'b.', label='Accepted(Final 50)',alpha=0.5)
ax.plot(rejected[-to_show:,0], rejected[-to_show:,1], 'rx', label='Rejected(Final 50)',alpha=0.5)
ax.set_xlabel("$\mu$")
ax.set_ylabel("$\sigma$")
ax.legend()


So, in the beginning (top plot), you see both the mean and sigma heading towards their optimal values, with some random-walk-like fluctuations along the way. The middle plot shows the entire evolution, with the initial jittery trajectory changing into a sort of circular orbit around the final parameters. Looking more closely at the end of the evolution in the bottom plot, we see apparently random fluctuations of both the accepted and rejected values, with the former more closely bunched around the best-fit point $[\mu, \sigma]=[50,20]$.


<h3>Autocorrelation</h3>

Another critical diagnostic when running Markov Chain Monte Carlo is how we can measure the convergence of the parameters in the Markov Chain. The simplest way to do this is to look at the spread of the parameters compared to their final result. If the parameters are spread around randomly (as appears to be the case in the bottom plot above), then their correlation coefficient will be close to zero. If the parameters are biased to one side, i.e. the parameter updates are mostly going in one specific direction (as is clearly the case in the top plot above), then there will be a strong correlation in the trend of the parameters.

To examine this quantitatively, we can define the autocorrelation for parmater $x_{i}$ in its i-th iteration as

$$
\hat{C}_{i} = \left(x_{i}-\bar{x}\right)\left(x_{i+n}-\bar{x}\right)
$$

where $n$ is the step size, namely how far apart in iteration number the points we compare are (typical length can be 100). For all practical purposes, this is just the correlation of an event with a previous event that is separated by some number of steps. Moreover, in the ideal scenario we should see that the autocorrelation follows a falling exponential distribution

$$
\hat{C}_{i} = e^{\large -t/\tau}
$$

with the approach to convergence indicated by an autocorrelation tending towards 0.

In [ ]:
#>>>RUN: L22.2-runcell02

mean_acc_mu=accepted[-100:,0].mean()
mean_acc_sig=accepted[-100:,1].mean()
print(mean_acc_mu,mean_acc_sig)

def autocorr(accepted,lag):
    num_mu=0
    denom_mu=0
    num_sig=0
    denom_sig=0
    rk_mu,rk_sig = 0, 0
    for i in range(accepted.shape[0]-lag):
        num_mu+=(accepted[i,0]-mean_acc_mu)*(accepted[i+lag,0]-mean_acc_mu)
        num_sig+=(accepted[i,1]-mean_acc_sig)*(accepted[i+lag,1]-mean_acc_sig)
        denom_mu+=(mean_acc_mu-accepted[i,0])**2
        denom_sig+=(mean_acc_sig-accepted[i,1])**2
    if denom_mu > 0 and denom_sig > 0:
        rk_mu=num_mu/denom_mu
        rk_sig=num_sig/denom_sig
    return rk_mu, rk_sig


lag=np.arange(1,100)
result=np.zeros((2,lag.shape[0]))
for l in lag:
    result[:,l-1]=autocorr(accepted,l)


fig, ax = plt.subplots()
ax.plot(lag, result[1,:], label='Autocorrelation for $\sigma$')
ax.plot(lag, result[0,:], label='Autocorrelation for $\mu$')
ax.legend(loc=0)
ax.set(xlabel='steps', ylabel='autocorrelation', ylim=(-1, 1))

This plot displays more quantitatively how $\sigma$ converges a bit more quickly than $\mu$, an effect which can be seen qualitatively in the plot we made above showing the entire evolution in 2D $\mu\sigma$ space.

Finally, perhaps the most critical element of the Markov Chain MC is the ability to investigate the correlations in the posterior distribution, which reveals the detailed behavior of the fit variables. We looked at something like this previously by plotting contours of the Likelihood. However, when things are not differentiable or there are ambiguities in the fit. Markov Chain posteriors can help to resolve difficult ambiguities in the fit.

In [ ]:
#>>>RUN: L22.2-runcell03
 
labels = ['$\mu$','$\sigma$']
print(accepted.shape)
samples=accepted[-5000:-1,:]
#samples=np.reshape(samples,(samples.shape[0]*samples.shape[1],samples.shape[2]))
#print(samples.shape)
fig = corner.corner(samples,show_titles=True,labels=labels,plot_datapoints=True,quantiles=[0.16, 0.5, 0.84])


In this so-called "corner" plot, we see both the distribution of the fit parameters, with the widths of the 1D histograms giving their uncertainties, and also whether or not the fit parameters have any correlation. In this case, the lack of any slope in the contours in the central figure indicates that $\sigma$ and $\mu$ are essentially totally uncorrelated.

<a name='exercises_23_2'></a>     

| [Top](#section_23_0) | [Restart Section](#section_23_2) | [Next Section](#section_23_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 22.2.1</span>

Plot the corner plot for the first 100 steps of the fit. What do your parameters look like, and do you observe a correlation (why or why not)?

A) The fitted parameters are similar to those in `L22.2-runcell03`, and can be used. There is no correlation. \
B) The equilibration stage is present in the corner plots, and you see parameters move to the best fit with a positive correltion. \
C) There is a negative correlation between the parameters.
<br>

In [ ]:
#>>>EXERCISE: L22.2.1

samples=#YOUR CODE HERE
fig = corner.corner(samples,show_titles=True,labels=labels,plot_datapoints=True,quantiles=[0.16, 0.5, 0.84])

<a name='section_23_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L22.3 A More Realistic Markov Chain MC</h2>  

| [Top](#section_23_0) | [Previous Section](#section_23_2) | [Exercises](#exercises_23_3) | [Next Section](#section_23_4) |

*The material in this section is discussed in the videos **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.3x+3T2023/block-v1:MITxT+8.S50.3x+3T2023+type@sequential+block@seq_LS22/block-v1:MITxT+8.S50.3x+3T2023+type@vertical+block@vert_LS22_vid3" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Overview</h3>

In this section, we are going to use an existing online dataset for the historical temperature of the earth over the past 800,000 years, but tweak it heavily to motivate MCMC. First, we need to load the data.

The citation for this temperature record can be found in at the beginning of this notebook, and the full data can be found <a href="https://www.ncei.noaa.gov/pub/data/paleo/icecore/antarctica/epica_domec/edc3deuttemp2007.txt" target="_blank">here</a>.

The horizontal axis is the age in years, with the zero at 1950.

In [ ]:
#>>>RUN: L22.3-runcell01

import csv 

def load(iFile='data/L23/ice_core_data.txt'):
    times=np.array([])
    amps =np.array([])
    with open(iFile, newline='') as csvfile:
        line = csv.reader(csvfile, delimiter='\t')
        for row in line:
            arr=row[0].split()
            #print(', '.join(row))
            pT=float(arr[2])
            pA=float(arr[4])
            times=np.append(pT,times)
            amps =np.append(pA,amps)
    #amps  = (amps-amps.mean())/(2.*np.max(amps))
    return times,amps

times,amps=load()
plt.plot(times,amps)
plt.xlabel('time (years)')
plt.ylabel('temp (C)')
plt.show()

data = np.vstack((times,amps))
data = data.T

This data has lots of big variations which may or may not have some periodic features. What we are going to do is make an arbitrary guess that there are three different cycles present. To investigate that, we'll try to fit the data with the following function (note the offset $A_0$ which is needed because the data are not centered vertically around 0):

$$
T(t) = A_{1} \sin \left(\omega_{1} t \right) + A_{2} \sin \left(\omega_{2} t \right) + A_{3} \sin \left(\omega_{3} t \right) + A_{0}
$$

We can begin by simply writing out the likelihood and running the MCMC as we did before. Notice the additions of a prior which requires the three frequencies to be in order from smallest to largest.


In [ ]:
#>>>RUN: L22.3-runcell02

def func(x,t):
    return x[0]*np.sin(x[1]*t) +  x[2]*np.sin(x[3]*t) +  x[4]*np.sin(x[5]*t) + x[6]

def log_like(x,data):
    return -1.*np.sum((func(x,data[:,0])-data[:,1])**2)

def prior(like,x):
    #Adjust the likelihood by the prior
    prior=0
    if(x[1] < x[3]) or (x[3] < x[5]) or (x[1] < x[5]):
        prior=1
    return like+np.log(prior)

def proposal(x,n=5000):
    #x[0] = mu, x[1]=sigma (new or current)
    x_new     = x.copy()
    #x_new[0]  = x[0] + np.random.randn()*x[0]/np.sqrt(n)
    #x_new[1]  = x[1] + np.random.randn()*x[1]/np.sqrt(n)
    #x_new[2]  = x[2] + np.random.randn()*x[0]/np.sqrt(n)
    #x_new[3]  = x[3] + np.random.randn()*x[1]/np.sqrt(n)
    #x_new[4]  = x[4] + np.random.randn()*x[0]/np.sqrt(n)
    #x_new[5]  = x[5] + np.random.randn()*x[1]/np.sqrt(n)
    #x_new[5]  = x[6] + np.random.randn()*x[1]/np.sqrt(n)
    x_new      = x + np.random.normal(x.shape[0])*(x*0.0001+1e-6)
    #np.random.normal(loc=0,scale=1,size=x.shape[0])*(x+1.0*np.ones(x.shape[0]))
    return x_new

def metropolis_hastings(iLikelihood,iPrior,iProposal,iAcceptance,xinit,data,niterations):
    x = xinit
    accepted = []
    rejected = []
    likelihood = []
    for i in range(niterations):
        x_new   =  iProposal(x)  
        likeOld = iLikelihood(x,data)
        likeNew = iLikelihood(x_new,data) 
        if (iAcceptance(iPrior(likeOld,x),iPrior(likeNew,x_new))):            
            x = x_new
            accepted.append(x_new)
            likelihood.append(likeNew)
        else:
            rejected.append(x_new)
    accepted = np.array(accepted)
    rejected = np.array(rejected)
    likelihood = np.array(likelihood)
    return accepted, rejected, likelihood

xinit = np.array([5.,1./10000.,5.,1./5000.,-5.,1./1000.,-5.])
accepted, rejected, likelihood = metropolis_hastings(log_like,prior,proposal,acceptance,xinit,data,15000)
print(accepted)

The last line in the code above prints out the entire array of accepted parameter values. Running 15,000 iterations results in only one case which is accepted.

Now, let's look at the likelihood "convergence" and what the fit looks zooming in on a small section of the date near the oldest samples.

In [ ]:
#>>>RUN: L22.3-runcell03

plt.plot(likelihood)
plt.xlabel("iteration")
plt.ylabel("$\mathcal{L}$")
plt.show()

#print(accepted[-1])
plt.plot(times[0:400],amps[0:400])
plt.plot(times[0:400],func(xinit,times)[0:400])
plt.xlabel('time (years)')
plt.ylabel('temp (C)')
plt.show()


The "convergence" is meaningless when only one iteration is accepted and, as a result, it's not surprising that the "fit" looks nothing like the data. In the previous example, the proposal function to find new values of the parameters varies each parameter by a Gaussian with mean and sigma of 0 and 1, respectively, times the current parameter values divided by 10,000. that clearly didn't work at all, so let's instead vary each parameter by that same Gaussian times 0.001. Note that this example runs 150,000 iterations instead of 15,000, and prints out only the total number of accepted iterations.

In [ ]:
#>>>RUN: L22.3-runcell04

def proposal(x,n=5000):
    return x + np.random.normal(x.shape[0])*1e-3

def prior(like,x):
    #Adjust the likelihood by the prior
    prior=0
    if(x[1] < x[3]) or (x[3] < x[5]) or (x[1] < x[5]):
        prior=1
    return like+np.log(prior)
    
def func(x,t):
    return x[0]*np.sin(x[1]/10000.*t) +  x[2]*np.sin(x[3]/10000*t) +  x[4]*np.sin(x[5]/10000*t) + x[6]

xinit = np.array([5.,1.,5.,2.,-5.,10.,-5.])
accepted, rejected, likelihood = metropolis_hastings(log_like,prior,proposal,acceptance,xinit,data,150000)
print(len(accepted))

plt.plot(likelihood)
plt.xlabel("iteration")
plt.ylabel("$\mathcal{L}$")
plt.show()

plt.plot(times[0:400],amps[0:400])
plt.plot(times[0:400],func(xinit,times)[0:400])
plt.xlabel('time (years)')
plt.ylabel('temp (C)')
plt.show()


Well, that looks a tiny bit better, with 18 accepted iterations (but out of 10 times more samples) and some evidence for a little bit of improvement in the likelihood. However, we are obviously not getting anywhere near to fitting the data.

<h3>A More Professional MCMC Sampler</h3>

This clearly is a very inefficient way to get to convergence. In light of that, let's see what the professional MCMC samplers actually do. Their strategy is to get closer to the traditional gradient descent methods that we typically perform when trying to fit data when things are differentiable.

The strategy will be to make the minimum searching a little more organized. To do that we will :

1. Search many directions in parallel (call each point a walker)
    * Perform a randomized update on all of the directions

2. Do a more organized stepping relying on the random gradient between two walkers
    * In this case we randomly pair walkers and search along the line connecting them

3. As more walkers converge to the best fit, the random selection pulls more walkers in
  
As a consequence of this, we will define the gradient update for this MCMC method using the existing points built with random selection. We can define this gradient as follows:

  * Given points $X_{i} \in X$, we pick randomly two points $X_{i}$ (our anchor) and $X_{j}$ (a second random point) and we define a random number $Z$, which is greater than 0.
  * Use $Z$ to pick a new point which is along the line between $X_{i}$ and $X_{j}$ given by (for the $t$-th and $(t+1)$-th step):

$$
X_{i}^{t+1} = X_{j}^{t} + Z (X^{t}_{i}-X^{t}_{j})
$$
  * Note that $X_{i}^{t+1}$ could be between $X_{i}^{t}$ and $X_{j}^{t}$, but also could be on the same line but outside those two points.
  * Now, we test the likelihood of $X_{i}^{t+1}$. If it's better that the likelihood of $X_{i}^{t}$, we throw out $X_{i}^{t}$ and keep $X_{i}^{t+1}$ as a new walker.
  * With the points divided into fixed and floating points, we match all the pairs and continually update our fixed points to be locations with better likelihood.

<h3>Example</h3>

As an example, the following code generates a set of toy data randomly sampled from a Gaussian with a mean of 10 and width of 2. Then, we follow these steps:

 * Create 10 random parameters (the Walkers)
 * Compute the p-values for our 10 walkers
 * Split the walkers into two parts (`walkers` and `otherwalkers`):
     * For each part we fixed `otherwalkers` and updated `walkers` by picking a random point along the line between a particular walker and a random otherwalker
     * Once we updated, we ran our traditional MCMC sampling
 * Then, we repeat the above steps but now varying the `otherwalkers` while the `walkers` were held fixed



In [ ]:
#>>>RUN: L22.3-runcell05

def mc_update(walkers, otherwalkers, logp0, Npars, idata, ifunc, a=2.0,iPrint=False):
    Nw = len(walkers)
    No = len(otherwalkers)

    # Calculates random gradient for the affine linear transformation 
    Z = (((a - 1.0) * np.random.rand(Nw) + 1.0) ** 2.0) / a
    if iPrint:
        print("Rand:",Z)
    
    # gets random indices of the othe rwalkers
    rint = np.random.randint(No, size=(Nw,))
    if iPrint:
        print("Rand Indx:",rint)
    
    # Propose new positions from the linear transformation
    qt1 = otherwalkers[rint] - Z[:, np.newaxis] * (otherwalkers[rint] - walkers)
    if iPrint:
        print("Rand Updates:", qt1)
    
    # Calculate the posterior probability of the new position
    logp1 = log_like(qt1, idata,ifunc)
    #logp1 = prior(logp1,qt1)
    
    #Now do the usual Markov update
    #p_diff = (Npars - 1) * np.log(Z) + logp - logp0
    p_diff = logp1 - logp0

    rshape=np.random.rand(p_diff.shape[0])
    # Determine if the new positions are accepted
    accept = p_diff > np.log(np.random.rand(p_diff.shape[0]))
    return qt1, logp1, accept

#now let's solve a 1 parameter problem
#try to predict a gaussian centered at 10
def tmpfunc(x,t):
    out=[]
    for pX in x:
        val=pX*np.ones(t.shape)
        out.append(val)
    out = np.array(out)
    return out

def log_like(x,data,ifunc):
    delta = ifunc(x,data[:,0])-data[:,1]*np.ones(x.shape)
    return -1.*np.sum(delta**2,axis=1)

Nwalkers=10
Npars=1
toydata  = np.random.normal(10,2,(10,2))
allWalks = np.array([1 + np.random.randn(Npars) for i in range(Nwalkers)])
logp0    = log_like(allWalks, toydata,tmpfunc)

print("Inital Walks:\n",allWalks,"\nInitial p-vals:\n",logp0)
print("Update first 5:")
newWalks, newlogp, accept = mc_update(allWalks[0:5], allWalks[5:10], logp0[0:5], Npars, toydata, tmpfunc,iPrint=True)
print("Updated Walks:\n",newWalks,"\nUpdated p-vals:\n",newlogp)

print("Update second 5:")
newWalks, newlogp, accept = mc_update(allWalks[5:10], allWalks[0:5], logp0[5:10], Npars, toydata, tmpfunc)
print("Updated Walks:\n",newWalks,"\nUpdated p-vals:\n",newlogp)


In the printout, you see the initial parameters and their p-values. The list labeled `Rand` is the $Z$ values used to find new points along the line between two parameter values in the formula shown earlier, `Rand Indx` indicates which random `walker` each `otherwalker` is paired with, `Rand Updates` are the new parameter values, and finally these new `walkers` and their new p-values are printed out. In the second step where the role of the `walkers` and `otherwalkers` are reversed, only the updated parameter values and their p-values are printed out.
 

All in all, our strategy here is to just do parallel MCMC updates on a grid of points to progressively get closer to a match to the sample. Because we have many points, we can take advantage of them crawling all over our distribution to get to the minimum.

Note that this walker strategy was developed fairly recently and has had its main use in astrophysics: see <a href="https://arxiv.org/abs/1710.06068" target="_blank">here</a>. The actual walker was developed by two statisticians (Goodman and Weare) who have worked closely with the astro community.  

Now that we've shown a simple example of how this works, let's do this for our more complicated problem of fitting the temperature data with a sum of three cyclical functions. We will only need about 2.5k steps to get to convergence, so let's just do that. 

In [ ]:
#>>>RUN: L22.3-runcell06

def proposal(q, logp, data, Nwalk, Npars, iFunc):
    half = int(Nwalk / 2)
    first, second = slice(half), slice(half, Nwalk)

    # Alternate slices of the data fixing and updating
    for S0, S1 in [(first, second), (second, first)]:
        # Use stretch move to calculate the proposal
        q_new, logp_new, acc = mc_update(q[S0], q[S1], logp[S0], Npars, data, iFunc)

        # Add accepted values into the chains
        if np.any(acc):
            logp[S0][acc] = logp_new[acc]
            q[S0][acc]    = q_new[acc]

    return q, logp

#def metropolis_hastings(iLikelihood,iPrior,iProposal,iAcceptance,xinit,data,niterations):
def metropolis_hastings_ensemble(xinit, logpinit, data, Npars, Nwalk, Nstep,iFunc):
    samples = np.ndarray((Nwalk, Nstep, Npars))
    samples[:, 0, :] = np.array(xinit)

    lnprob = np.ndarray((Nwalk, Nstep))
    lnprob[:, 0] = np.array(logpinit)

    # Iterate over the Markov steps
    for i in range(1, Nstep):
        if (i % 500)==0:
            print("Steps:",i)
        q, p = proposal(samples[:,i-1,:], lnprob[:,i-1], data, Nwalk, Npars, iFunc)
        samples[:,i,:] = np.array(q)
        lnprob[:,i] = np.array(p)
    
    return samples, lnprob

# Set up MCMC parameters
Npars = xinit.shape[0]
Nwalk = 100
Nstep = 2500 #larger Nstep used in related video

def log_like(x,data,iFunc):
    return -1.*np.sum((iFunc(x,data[:,0])-data[:,1])**2,axis=1)

def func(x,t):
    out=[]
    for pX in x:
        val=pX[0]*np.sin(pX[1]/10000.*t) +  pX[2]*np.sin(pX[3]/10000*t) +  pX[4]*np.sin(pX[5]/10000*t) + pX[6]
        out.append(val)
    out = np.array(out)
    return out

q0    = np.array([xinit + 1.0e-4*np.random.randn(Npars) for i in range(Nwalk)])
logp0 = log_like(q0, data, func)
accepted, likelihood = metropolis_hastings_ensemble(q0,logp0,data,Npars,Nwalk,Nstep,func)

In [ ]:
#>>>RUN: L22.3-runcell07

lproj = np.max(likelihood,axis=0)
plt.plot(lproj)
plt.xlabel("iteration")
plt.ylabel("$\mathcal{L}$")
plt.show()

maxval=np.argmax(lproj)
maxy=np.argmax(likelihood[:,maxval])
bestpars=np.array([accepted[maxy,maxval]])
output=func(bestpars,times)

plt.plot(times[0:4000],amps[0:4000])
plt.plot(times[0:4000],output.flatten()[0:4000])
plt.xlabel('time (years)')
plt.ylabel('temp (C)')
plt.show()


This method works much better! The likelihood shows clear convergence to a maximum value and the fit to the data is much more reasonable. Clearly, this simple sum of three sinusoidal functions cannot capture every detail of this complicated data, but it does seem to accurately reproduce many of the features present.

Just for fun, let's look at the best fit of the final walkers. We can see that many of the features in the data are picked up form these walkers. 

In [ ]:
#>>>RUN: L22.3-runcell08

def plotter(accepted,times,amps):
    plt.ion()
    plt.plot(times,amps,label='Ice Core Data')
    output=func(accepted[:,-1],times)
    for i0 in range(100):
        plt.plot(times, output[i0], color="r", alpha=0.1)
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.xlabel('Years ago')
    plt.ylabel(r'$\Delta$ T (C)')
    plt.legend()
    plt.show()
    
plotter(accepted,times,amps)

Now, we can look in more detail at the parameters, which is really where the Markov Chain MC starts to show its stuff. Specifically, we'll plot the correlation of each parameter with all of the others. This will be done similarly to what we did previously, with "corner" plots showing 2D contours and 1D distributions.

In [ ]:
#>>>RUN: L22.3-runcell09

import corner 
labels = ['$a_{1}$','$\omega_{1}$','$a_{2}$','$\omega_{2}$','$a_{3}$','$\omega_{3}$','A']
samples=accepted[:,-500:-1]
samples=np.reshape(samples,(samples.shape[0]*samples.shape[1],samples.shape[2]))
print(samples.shape)
fig = corner.corner(samples,show_titles=True,labels=labels,plot_datapoints=True,quantiles=[0.16, 0.5, 0.84])


The contours and 1D plots show lots of ambiguity with multiple solutions for the various amplitudes and frequencies. Note that, unlike previously, this version didn't impose any prior constraint on the amplitudes. We can try again including the prior requirement.

In [ ]:
#>>>RUN: L22.3-runcell10

def prior(like,x):
    #Adjust the likelihood by the prior
    prior=np.ones(x.shape[0])
    idx = np.where((x[:,1] < x[:,3]) | (x[:,3] < x[:,5]) | (x[:,1] < x[:,5]))
    like[idx]+=np.log(prior[idx])
    return like

def mc_update(walkers, otherwalkers, logp0, Npars, idata, ifunc, a=2.0,iPrint=False):
    Nw = len(walkers)
    No = len(otherwalkers)

    # Calculates random gradient for the affine linear transformation 
    Z = (((a - 1.0) * np.random.rand(Nw) + 1.0) ** 2.0) / a
    rint = np.random.randint(No, size=(Nw,))
    qt1 = otherwalkers[rint] - Z[:, np.newaxis] * (otherwalkers[rint] - walkers)    
    # Calculate the posterior probability of the new position
    logp1 = log_like(qt1, idata,ifunc)
    logp1 = prior(logp1,qt1)
    
    #Now do the usual Markov update
    #p_diff = (Npars - 1) * np.log(Z) + logp - logp0
    p_diff = logp1 - logp0

    rshape=np.random.rand(p_diff.shape[0])
    accept = p_diff > np.log(np.random.rand(p_diff.shape[0]))
    return qt1, logp1, accept

Nwalk = 100
Nstep = 2500 #larger Nstep used in related video
q0    = np.array([xinit + 1.0e-4*np.random.randn(Npars) for i in range(Nwalk)])
logp0 = log_like(q0, data, func)
accepted, likelihood = metropolis_hastings_ensemble(q0,logp0,data,Npars,Nwalk,Nstep,func)

Let's see how the corner plots of correlations have changed with the prior included.

In [ ]:
#>>>RUN: L22.3-runcell11

import corner 
labels = ['$a_{1}$','$\omega_{1}$','$a_{2}$','$\omega_{2}$','$a_{3}$','$\omega_{3}$','A']
samples=accepted[:,-500:-1]
samples=np.reshape(samples,(samples.shape[0]*samples.shape[1],samples.shape[2]))
print(samples.shape)
fig = corner.corner(samples,show_titles=True,labels=labels,plot_datapoints=True,quantiles=[0.16, 0.5, 0.84])

plotter(accepted,times,amps)

lproj = np.max(likelihood,axis=0)
plt.plot(lproj)
plt.xlabel("iteration")
plt.ylabel("$\mathcal{L}$")
plt.show()

maxval=np.argmax(lproj)
maxy=np.argmax(likelihood[:,maxval])
bestpars=np.array([accepted[maxy,maxval]])
output=func(bestpars,times)

plt.plot(times[0:4000],amps[0:4000])
plt.plot(times[0:4000],output.flatten()[0:4000])
plt.xlabel('time (years)')
plt.ylabel('temp (C)')
plt.show()

The corner plots show a lot of scatter of individual points, but the peaks indicated by the contours are quite distinct, something that is more evident in the projected 1D distributions. With the exception of `a3`, which has a small satellite peak, all of the parameters have a single clear peak. Even in the case of `a3`, that extra peak doesn't move the mean value very much, but instead only results is an increased uncertainty on the low side.

In this particular case, the simple function we are using for the fit is differentiable, so we can compare the MCMC approach with a normal fitter. The time to convergence will be much faster, but this convergence will not be great... Well, we really don't even know how to characterize what's good and what's bad without a better notion of the uncertainty and correlation model in the data. Anyway let's take a look.


In [ ]:
#>>>RUN: L22.3-runcell12

xinit = np.array([2.,1.6,1.76,2.16,-0.1,10.6,-5.])

def lmfunc(x,p0,p1,p2,p3,p4,p5,p6):
    #val=pX[0]*np.sin(pX[1]/10000.*t) +  pX[2]*np.sin(pX[3]/10000*t) +  pX[4]*np.sin(pX[5]/10000*t) + pX[6]
    return p0*np.sin(p1/10000.*x) +  p2*np.sin(p3/10000.*x) +  p4*np.sin(p5/10000.*x) + p6

model  = lmfit.Model(lmfunc)
print(data[:,1],data[:,0])
params = model.make_params(p0=xinit[0],p1=xinit[1],p2=xinit[2],p3=xinit[3],p4=xinit[4],p5=xinit[5],p6=xinit[6])
result = model.fit(data=data[:,1], params=params, x=data[:,0])
result.plot()
lmfit.report_fit(result)

<a name='exercises_23_3'></a>     

| [Top](#section_23_0) | [Restart Section](#section_23_3) | [Next Section](#section_23_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 22.3.1</span>

Run the fit with only 6 walkers, but still 2500 steps. What is the value of the likelihood that the code converges to? Think about how this compares to the convergence of the earlier code, where 100 walkers were used. Is this doing a good enough job?

Run several trials and report your answer for the typical maximum likelihood as a number (it will be negative) with precision `1e5`.


<br>

In [ ]:
#>>>EXERCISE: L22.3.1

Nwalk = ###YOUR CODE HERE
Nstep = 2500
xinit = np.array([5.,1.,5.,2.,-5.,10.,-5.])
q0    = np.array([xinit + 1.0e-4*np.random.randn(Npars) for i in range(Nwalk)])
logp0 = log_like(q0, data, func)
accepted, likelihood = metropolis_hastings_ensemble(q0,logp0,data,Npars,Nwalk,Nstep,func)

lproj = np.max(likelihood,axis=0)
max_likelihood = ###YOUR CODE HERE

print("Maximum Liikelihood:", max_likelihood)
plt.plot(lproj)
plt.xlabel("iteration")
plt.ylabel("$\mathcal{L}$")
plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 22.3.2</span>

Let's run MCMC on some data that has discontinuities, and see if we can get a good fit using a step-function. There are several steps to understanding and implementing this. In this problem we will describe the data and fit function, with the goal of defining a prior.

<h3>Step 1: Generate Data</h3>

Generate three sets of random data, that will ultimately create a tiered box shape. Run the following code to visualize this shape:

<pre>
np.random.seed(40)
vals=np.random.rand(1000)*10
vals=np.append(vals,np.random.rand(1000)*5 + 2.5)
vals=np.append(vals,np.random.rand(1000)*2+4.)
hist,bin_edges=np.histogram(vals,bins=np.arange(0,10.5,0.25))
bin_centers=0.5*(bin_edges[:-1]+bin_edges[1:])
plt.errorbar(bin_centers,hist,np.sqrt(hist),fmt='o', color='k')
plt.xlabel("x")
plt.ylabel("events")
plt.show()
</pre>

<h3>Step 2: Define a Fit Function</h3>

The function we will try to fit is given by:

<pre>
def func(x,t):
    out=[]
    for pX in x:
        val=pX[0]*np.heaviside(t-pX[4],1) + \
            pX[1]*np.heaviside(t-pX[5],1) + \
            pX[2]*np.heaviside(t-pX[6],1) + \
            pX[3]*np.heaviside(t-pX[7],1) + pX[8]
        out.append(val)
    out = np.array(out)
    return out
</pre>

Optionally, plot the fit function for some random parameter choices, to see what it looks like:

<pre>
# Define a single set of parameters
# Format: [height1, height2, height3, height4, position1, position2, position3, position4, offset]
params = np.array([
    [2, -1, 3, -2, 1, 3, 5, 7, 0]  # Example parameter set
])

# Define the t values
t_values = np.linspace(0, 10, 1000)  # t goes from 0 to 10 with 1000 points

# Calculate the output of the function for this single set of parameters
output = func(params, t_values)

# Plot the result
plt.figure(figsize=(10, 6))
plt.plot(t_values, output[0], label="Parameter Set")
plt.xlabel('t')
plt.ylabel('Function Value')
plt.title('Plot of func for a Single Parameter Set')
plt.legend()
plt.grid(True)
plt.show()
</pre>


<h3>Step 3: Define a Prior</h3>

The prior places constraints on the parameters of the model. It will have the following form:

<pre>
def prior(like,x):
    #Adjust the likelihood by the prior
    prior=np.zeros(x.shape[0])
    idx = np.where(###YOUR CODE HERE: ENTER CONSTRAINTS)
    like[idx]+=np.log(prior[idx])
    
    #ADD OTHER CONSTRAINTS
    idx = np.where(###YOUR CODE HERE: ENTER CONSTRAINTS)
    like[idx]+=np.log(prior[idx])
    return like
</pre>


**Here is where the question comes in.** Consider the constraints below. Select ALL options that are beneficial to add to the prior, based on the form of the data and model we are trying to fit. Note: some options may seem useful, but are not ideal because

A) The step positions should be strictly increasing (i.e., each step occurs at a farther position than the previous one):\
`idx = np.where((x[:,4] > x[:,5]) | (x[:,5] > x[:,6]) | (x[:,6] > x[:,7]))`

B) The step heights should be non-negative, meaning the model should not have any downward steps:\
`idx = np.where((x[:,0] < 0) | (x[:,1] < 0) | (x[:,2] < 0) | (x[:,3] < 0))`

C) The difference between consecutive step positions should be at least 1 to ensure that the steps are well-separated:\
`idx = np.where((x[:,5] - x[:,4] < 1.0) | (x[:,6] - x[:,5] < 1.0) | (x[:,7] - x[:,6] < 1.0))`

D) The step positions should be non-negative, meaning all steps should occur at positive positions:\
`idx = np.where((x[:,4] < 0) | (x[:,5] < 0) | (x[:,6] < 0) | (x[:,7] < 0))`

E) The sum of the step heights should be equal to a specific value (e.g., 10) to ensure the total height of the function is fixed:\
`idx = np.where(np.abs(x[:,0] + x[:,1] + x[:,2] + x[:,3] - 10) > 1e-6)`

F) The step heights should alternate in sign, ensuring that each step is followed by a drop, creating an oscillating pattern:
`idx = np.where((x[:,0] * x[:,1] > 0) | (x[:,1] * x[:,2] > 0) | (x[:,2] * x[:,3] > 0))`

G) The step positions should be within a specific range (e.g., between 1 and 5) to limit the function’s domain:\
`idx = np.where((x[:,4] < 1) | (x[:,4] > 5) | (x[:,5] < 1) | (x[:,5] > 5) | (x[:,6] < 1) | (x[:,6] > 5) | (x[:,7] < 1) | (x[:,7] > 5))`

H) The constant offset should be non-negative to ensure that the function does not drop below a certain baseline:\
`idx = np.where(x[:,8] < 0)`


<br>

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 22.3.3</span>

Now run MCMC to fit the step-function that we have defined, and compare with fitting performed by lmfit. You will need to define the `prior` in the code below, based on your answer to the previous question. Which does a better job? 

A) MCMC is better.\
B) lmfit is better.\
C) They both perform the same.

<br>

In [ ]:
#>>>EXERCISE: L22.3.3

#Generate the data -> just run this code if 
#you want to see what it looks like first
#-----------------------------------------------
np.random.seed(40)
vals=np.random.rand(1000)*10
vals=np.append(vals,np.random.rand(1000)*5 + 2.5)
vals=np.append(vals,np.random.rand(1000)*2+4.)
hist,bin_edges=np.histogram(vals,bins=np.arange(0,10.5,0.25))
bin_centers=0.5*(bin_edges[:-1]+bin_edges[1:])
plt.errorbar(bin_centers,hist,np.sqrt(hist),fmt='o', color='k')
plt.xlabel("x")
plt.ylabel("events")
plt.show()


#define the step-function used for fitting
#-----------------------------------------------
def func(x,t):
    out=[]
    for pX in x:
        val=pX[0]*np.heaviside(t-pX[4],1) + \
            pX[1]*np.heaviside(t-pX[5],1) + \
            pX[2]*np.heaviside(t-pX[6],1) + \
            pX[3]*np.heaviside(t-pX[7],1) + pX[8]
        out.append(val)
    out = np.array(out)
    return out

def log_like(x,data,iFunc):
    return -1.*np.sum((iFunc(x,data[:,0])-data[:,1])**2,axis=1)


#define the prior
#-----------------------------------------------
def prior(like,x):
    #Adjust the likelihood by the prior
    prior=np.zeros(x.shape[0])
    idx = np.where(###YOUR CODE HERE: ENTER CONSTRAINTS)
    like[idx]+=np.log(prior[idx])
    
    #ADD OTHER CONSTRAINTS
    idx = np.where(###YOUR CODE HERE: ENTER CONSTRAINTS)
    like[idx]+=np.log(prior[idx])
    return like


#redefine some functions that we have used already
#-----------------------------------------------
def mc_update(walkers, otherwalkers, logp0, Npars, idata, ifunc, a=2.0,iPrint=False):
    Nw = len(walkers)
    No = len(otherwalkers)

    # Calculates random gradient for the affine linear transformation
    Z = (((a - 1.0) * np.random.rand(Nw) + 1.0) ** 2.0) / a
    rint = np.random.randint(No, size=(Nw,))
    qt1 = otherwalkers[rint] - Z[:, np.newaxis] * (otherwalkers[rint] - walkers)
    # Calculate the posterior probability of the new position
    logp1 = log_like(qt1, idata,ifunc)
    logp1 = prior(logp1,qt1)

    #Now do the usual Markov update
    #p_diff = (Npars - 1) * np.log(Z) + logp - logp0
    p_diff = logp1 - logp0

    rshape=np.random.rand(p_diff.shape[0])
    accept = p_diff > np.log(np.random.rand(p_diff.shape[0]))
    return qt1, logp1, accept


def log_like(x,data,iFunc):
    return -1.*np.sum((iFunc(x,data[:,0])-data[:,1])**2,axis=1)


#run MCMC
#-----------------------------------------------
Npars = 9
Nwalk = 100
Nstep = 5000
xinit = np.array([100,100,-100,-100,1,2,3,4,100])
q0    = np.array([xinit + 1.0e-4*np.random.randn(Npars) for i in range(Nwalk)])
data = np.vstack((bin_centers,hist))
data = data.T
logp0 = log_like(q0, data, func)
accepted, likelihood = metropolis_hastings_ensemble(q0,logp0,data,Npars,Nwalk,Nstep,func)

lproj = np.max(likelihood,axis=0)
plt.plot(lproj)
plt.xlabel("iteration")
plt.ylabel("$\mathcal{L}$")
plt.show()

maxval=np.argmax(lproj)
maxy=np.argmax(likelihood[:,maxval])
bestpars=np.array([accepted[maxy,maxval]])
print(bestpars)
output=func(bestpars,bin_centers)

plt.errorbar(bin_centers,hist,np.sqrt(hist),fmt='o', color='k')
plt.plot(bin_centers,output.flatten(),color='red')
plt.xlabel("x")
plt.ylabel("events")
plt.show()


#fit with lmfit
#-----------------------------------------------
def lmfunc(x,p0,p1,p2,p3,p4,p5,p6,p7,p8):
    return p0*np.heaviside(x-p4,1) + p1*np.heaviside(x-p5,1) + p2*np.heaviside(x-p6,1) + p3*np.heaviside(x-p7,1)+p8

model  = lmfit.Model(lmfunc)
print(data[:,1],data[:,0])
params = model.make_params(p0=xinit[0],p1=xinit[1],p2=xinit[2],p3=xinit[3],p4=xinit[4],p5=xinit[5],p6=xinit[6],p7=xinit[7],p8=xinit[8])
result = model.fit(data=data[:,1], params=params, x=data[:,0])
result.plot()
lmfit.report_fit(result)